In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D,GRU_NCL,skipConnectWrap1d

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [5]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [5]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [6]:
conv = Sequential(ConcatWrap1d(ConvBatchRelu1D(3,5,3,padding=1)),
                  Dropout(0.25),
                  ConcatWrap1d(ConvBatchRelu1D(8,8,3,padding=1)),
                  Dropout(0.25),
                  ConcatWrap1d(ConvBatchRelu1D(16,16,3,padding=1)),
                  Dropout(0.5),
                  ConcatWrap1d(ConvBatchRelu1D(32,32,3,padding=1)),
                  Dropout(0.5))

In [7]:
rnn = GRU_NCL(64,64,2,batch_first=True,dropout=0.5,bidirectional=True)

In [8]:
seq_model = Sequential(conv,rnn)

In [9]:
linear = Sequential(Linear(128,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [10]:
convert = Sequential(Linear(120,60),LeakyReLU(0.1,True),Dropout(0.5))

In [11]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized_NCL(seq_model,linear,convert).to('cuda:0')

In [12]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [13]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [14]:
model_nor = fit(5, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation

In [25]:
model_nor = fit(1, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

epoch:0, train_loss:0.777988509501818, val_loss:0.8216824531555176
Training completed in 44.527931928634644s


In [26]:
model = CNN_RNN2seq_logNormalized_NCL(seq_model,linear,convert).to('cuda:0')

In [27]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [28]:
loss_func = loss_func_generator(SMAPE)

In [29]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.88876694694084, val_loss:36.922828674316406
epoch:1, train_loss:37.79922653059885, val_loss:36.890682220458984
epoch:2, train_loss:37.71661737965796, val_loss:36.64965057373047
Training completed in 158.67075490951538s


In [30]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.58995795867604, val_loss:36.603302001953125
epoch:1, train_loss:37.543538613393515, val_loss:36.41912078857422
epoch:2, train_loss:37.499908486796166, val_loss:36.63462829589844
Training completed in 159.22054409980774s


In [31]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.42186022150702, val_loss:36.4130973815918
epoch:1, train_loss:37.43719141075648, val_loss:36.77753448486328
epoch:2, train_loss:37.41057199665919, val_loss:36.195472717285156
Training completed in 160.05372047424316s


In [32]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.33798809051514, val_loss:36.38883590698242
epoch:1, train_loss:37.36783977963146, val_loss:36.219600677490234
epoch:2, train_loss:37.27405398670256, val_loss:36.03410720825195
Training completed in 159.25932550430298s


In [33]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.281399445212564, val_loss:36.45156478881836
epoch:1, train_loss:37.24218135596557, val_loss:36.0681037902832
epoch:2, train_loss:37.23800328605533, val_loss:35.887428283691406
Training completed in 159.996484041214s
